# Actividad 5:
# Comparación de estrategias de tuning automático para modelos de clasificación en salud

## Objetivo
Aplicar técnicas de tuning automatizado con las librerías Optuna y Ray Tune para mejorar el rendimiento de un modelo de clasificación binaria (Random Forest) sobre un conjunto de datos médicos. El alumno deberá comparar ambas herramientas y reflexionar sobre su efectividad.

**Dataset utilizado:**  
Cáncer de mama de Scikit-learn

---

### Estructura del Notebook:
1. Metodología.
2. Configuración inicial del notebook.
3. Definicion de funciones.
4. Uso de funciones y resultados.
5. Análisis de los resultados y reflexiones finales.

---

## 1. Metodología

---

### Flujo de trabajo
1. **Carga de datos:**
   - Escalado de características con `StandardScaler`.
   - División de datos en entrenamiento y prueba (70% / 30%) con estratificación de clases.

2. **Modelos creados:**
   - **Modelo base:** RandomForestClassifier con hiperparámetros por defecto.
   - **Modelo optimizado con Optuna:** RandomForestClassifier optimizado mediante Optuna con 20 trials.
   - **Modelo optimizado con Ray Tune:** RandomForestClassifier optimizado mediante Ray Tune con 20 trials. Uso de CallBack para mostrar logs.

3. **Evaluación de modelos:**
   - **Métricas evaluadas:**
      - F1 Score.
      - Tiempo de ejecución.

4. **Comparación y visualización:**
   - Resumen del dataset.
   - Mejores hiperparametros encontrados por Optuna y Ray Tune.
   - Resultados del modelo base, optimizado con Optuna y optimizado con Ray Tune.
   - Gráficos de comparación de rendimiento (F1-Score) y tiempos de ejecución.
   - Gráficos de evolución de F1-Score en los modelos optimizados con Optuna y Ray Tune.
   - Matrices de confusión del modelo base, Optuna y Ray Tune.

---

# 2. Configuración inicial del notebook
- Importación de librerias necesarias.

--- 

In [ ]:
import pandas as pd
import numpy as np
import time
import matplotlib.pyplot as plt
import seaborn as sns
import logging
import os
from sklearn.datasets import load_breast_cancer
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import f1_score, confusion_matrix
import optuna
import ray
from ray import train, tune
from ray.tune import Tuner
from ray.tune.search.basic_variant import BasicVariantGenerator
from ray.tune import Callback
from datetime import datetime

# 3. Definición de funciones

> **Nota:** Para mejor comprensión de las funciones y su utilidad, esta sección se divide en bloques, en donde cada uno responde a una parte diferente de la metodología de trabajo. 

---

**Bloque 0:** Configuración del entorno

- **`setup_environment()`** 
Debido a lso multiples errores y warnings que ocurren durante la ejecución, se decidió configurar el setup para mejorar la legibilidad de las salidas de codigo.

In [ ]:
def setup_environment():
    """
    Configura variables de entorno, logging y estilo gráfico.

    Incluye:
    - Desactivación de servicios no usados por Apache Arrow.
    - Desactivación del dashboard de Ray.
    - Activación de captura de logs en stderr.
    - Reducción de verbosidad en los logs.
    - Estilo gráfico para visualizaciones (seaborn y matplotlib).

    Esta función se debe llamar al inicio del flujo principal.
    """
    # Configuraciones avanzadas para evitar problemas con Ray
    os.environ['ARROW_DISABLE_S3'] = '1'
    os.environ['ARROW_DISABLE_GCS'] = '1'
    os.environ['ARROW_DISABLE_HDFS'] = '1'
    os.environ['RAY_DISABLE_DASHBOARD'] = '1'
    os.environ['RAY_ENABLE_WINDOWS_OR_OSX_CLUSTER'] = '1'
    os.environ['RAY_LOG_TO_STDERR'] = '1'  # Ahora capturaremos estos logs
    os.environ['RAY_DEDUP_LOGS'] = '0'  # Mostrar todos los logs
    
    # Reducir verbosidad de Arrow
    os.environ['ARROW_VERBOSE_THRESHOLD'] = '0'
    
    # Configuración de logging
    logging.basicConfig(level=logging.ERROR)
    
    # Configurar estilo para gráficos
    sns.set_style("whitegrid")
    plt.rcParams['figure.figsize'] = (12, 6)
    plt.rcParams['font.size'] = 12
    plt.rcParams['figure.max_open_warning'] = 50  # Permitir más figuras

**Bloque 1:** Carga y preprocesamiento de datos

- **`load_and_preprocess_data()`** 
Carga el dataset de cáncer de mama, aplica escalado estándar y divide los datos en conjuntos de entrenamiento y prueba estratificados.

In [ ]:
def load_and_preprocess_data(data):
    """
    Realiza la carga y el preprocesamiento del dataset de cáncer de mama.

    Este paso incluye:
    - División del conjunto de datos en entrenamiento y prueba (70/30, con estratificación).
    - Escalamiento de características numéricas usando StandardScaler.
    - Creación de un resumen informativo del dataset.

    Args:
        data (sklearn.utils.Bunch): Objeto cargado con `load_breast_cancer()` de scikit-learn.

    Returns:
        tuple:
            - X_train_scaled (np.ndarray): Datos de entrenamiento escalados.
            - X_test_scaled (np.ndarray): Datos de prueba escalados.
            - y_train (np.ndarray): Etiquetas de entrenamiento.
            - y_test (np.ndarray): Etiquetas de prueba.
            - data (Bunch): El dataset original sin modificar.
            - info_df (pd.DataFrame): DataFrame resumen del dataset (tamaño, clases, particiones, etc.).
    """
    
    # Cargar dataset de cáncer de mama
    # data = load_breast_cancer()
    X = data.data
    y = data.target
    
    # Dividir datos en entrenamiento y prueba
    X_train, X_test, y_train, y_test = train_test_split(
        X, y, test_size=0.3, random_state=42, stratify=y
    )
    
    # Escalar características
    scaler = StandardScaler()
    X_train_scaled = scaler.fit_transform(X_train)
    X_test_scaled = scaler.transform(X_test)

    # Crear resumen en un DataFrame
    info_dict = {
        "Total muestras": [X.shape[0]],
        "N° características": [X.shape[1]],
        "Clases": [", ".join(data.target_names)],
        "Distribución clases": [f"{np.bincount(y)[0]} (malignant), {np.bincount(y)[1]} (benign)"],
        "Train size": [X_train.shape[0]],
        "Test size": [X_test.shape[0]],
        "Shape X_train": [X_train_scaled.shape],
        "Shape X_test": [X_test_scaled.shape]
    }
    info_df = pd.DataFrame(info_dict)

    return X_train_scaled, X_test_scaled, y_train, y_test, data, info_df

**Bloque 2:** Aplicación del modelo base, y optimizados con Optuna y Ray Tune

- **`train_baseline_model()`** 
Entrena y evalúa un modelo base Random Forest con hiperparámetros por defecto.

- **`optimize_with_optuna()`** 
Realiza optimización de hiperparámetros para un modelo Random Forest usando Optuna.

- **`optimize_with_ray_tune()`** 
Realiza optimización de hiperparámetros para un modelo Random Forest utilizando Ray Tune

In [ ]:
def train_baseline_model(X_train, y_train, X_test, y_test, target_names):
    """
    Entrena y evalúa un modelo base Random Forest con hiperparámetros por defecto.

    El modelo se ajusta sobre los datos de entrenamiento y se evalúa usando F1-score
    sobre el conjunto de prueba.

    Args:
        X_train (np.ndarray): Conjunto de entrenamiento (features) escalado.
        y_train (np.ndarray): Etiquetas del conjunto de entrenamiento.
        X_test (np.ndarray): Conjunto de prueba (features) escalado.
        y_test (np.ndarray): Etiquetas del conjunto de prueba.
        target_names (np.ndarray): Nombres de las clases objetivo (no usado directamente aquí, 
                                   pero útil para extensiones o reportes detallados).

    Returns:
        tuple:
            - base_model (RandomForestClassifier): Modelo entrenado.
            - base_f1 (float): F1-score en el conjunto de prueba.
            - base_train_time (float): Tiempo de entrenamiento (segundos).
            - y_pred_base (np.ndarray): Predicciones del modelo base en el test set.
    """
    # Entrenar modelo con parámetros por defecto
    start_time = time.time()
    base_model = RandomForestClassifier(random_state=42)
    base_model.fit(X_train, y_train)
    base_train_time = time.time() - start_time

    # Evaluación del modelo
    y_pred_base = base_model.predict(X_test)
    base_f1 = f1_score(y_test, y_pred_base)

    return base_model, base_f1, base_train_time, y_pred_base


def optimize_with_optuna(X_train, y_train, X_test, y_test, n_trials=20):
    """
    Realiza optimización de hiperparámetros para un modelo Random Forest usando Optuna.

    Busca maximizar el F1-score en el conjunto de prueba, evaluando distintos 
    conjuntos de parámetros mediante validación directa. Al finalizar, se 
    entrena un modelo final con los mejores parámetros encontrados.

    Args:
        X_train (np.ndarray): Conjunto de entrenamiento (features) escalado.
        y_train (np.ndarray): Etiquetas del conjunto de entrenamiento.
        X_test (np.ndarray): Conjunto de prueba (features) escalado.
        y_test (np.ndarray): Etiquetas del conjunto de prueba.
        n_trials (int, opcional): Número de combinaciones a probar (default: 20).

    Returns:
        tuple:
            - optuna_best_model (RandomForestClassifier): Modelo final entrenado con los mejores parámetros.
            - optuna_f1 (float): F1-score del modelo optimizado en el test set.
            - optuna_time (float): Tiempo total de optimización en segundos.
            - optuna_results (pd.DataFrame): Detalles de todos los trials realizados.
            - y_pred_optuna (np.ndarray): Predicciones del modelo optimizado.
            - optuna_best_params (dict): Diccionario con los mejores hiperparámetros encontrados.
            - best_trial (int): Número del mejor trial según F1-score.
    """
    print(f"\nOptimizando modelo con Optuna ({n_trials} trials)")
    
    # Función objetivo para Optuna
    def objective(trial):
        # Espacio de búsqueda
        params = {
            'n_estimators': trial.suggest_int('n_estimators', 10, 200),
            'max_depth': trial.suggest_int('max_depth', 2, 32),
            'min_samples_split': trial.suggest_int('min_samples_split', 2, 20)
        }
        
        # Crear y entrenar modelo
        model = RandomForestClassifier(**params, random_state=42)
        model.fit(X_train, y_train)
        
        # Evaluar con F1-score
        y_pred = model.predict(X_test)
        return f1_score(y_test, y_pred)

    # Ejecutar estudio de optimización
    print("Iniciando búsqueda de hiperparámetros con Optuna...")
    start_time = time.time()
    study = optuna.create_study(direction='maximize')
    study.optimize(objective, n_trials=n_trials)
    optuna_time = time.time() - start_time

    # Obtener mejores parámetros
    optuna_best_params = study.best_params
    optuna_best_model = RandomForestClassifier(**optuna_best_params, random_state=42)
    optuna_best_model.fit(X_train, y_train)
    y_pred_optuna = optuna_best_model.predict(X_test)
    optuna_f1 = f1_score(y_test, y_pred_optuna)

    # Resultados de los trials
    optuna_results = study.trials_dataframe()
    
    return optuna_best_model, optuna_f1, optuna_time, optuna_results, y_pred_optuna, optuna_best_params, study.best_trial.number


def optimize_with_ray_tune(X_train, y_train, X_test, y_test, n_trials=20):
    """
    Realiza optimización de hiperparámetros para un modelo Random Forest utilizando Ray Tune.

    Usa búsqueda aleatoria (BasicVariantGenerator) sobre un espacio de hiperparámetros definido.
    Registra el F1-score de cada configuración y retorna el modelo con mejor desempeño, junto a
    sus resultados detallados. Incluye captura y despliegue de logs personalizados durante el proceso.

    Args:
        X_train (np.ndarray): Conjunto de entrenamiento (features) escalado.
        y_train (np.ndarray): Etiquetas del conjunto de entrenamiento.
        X_test (np.ndarray): Conjunto de prueba (features) escalado.
        y_test (np.ndarray): Etiquetas del conjunto de prueba.
        n_trials (int, opcional): Número de configuraciones a evaluar (default: 20).

    Returns:
        tuple:
            - ray_best_model (RandomForestClassifier): Modelo final entrenado con los mejores parámetros.
            - ray_f1 (float): F1-score del modelo optimizado en el test set.
            - ray_time (float): Tiempo total de optimización en segundos.
            - ray_results_df (pd.DataFrame): DataFrame con el resumen de cada trial.
            - y_pred_ray (np.ndarray): Predicciones del modelo optimizado.
            - ray_best_params (dict): Hiperparámetros óptimos encontrados.
    """
  
    # Callback para mostrar progreso
    class ProgressCallback(Callback):
        def __init__(self, total_trials):
            self.total_trials = total_trials
            self.completed_trials = 0
            self.header_printed = False

        def on_trial_complete(self, iteration, trials, trial):
            if not self.header_printed:
                print("="*50)
                print(f"Optimizando modelo con Ray Tune ({self.total_trials} trials)")
                print("="*50)
                self.header_printed = True

            self.completed_trials += 1
            f1 = trial.last_result['f1_score']
            params = trial.config
            timestamp = datetime.now().strftime("%Y-%m-%d %H:%M:%S")
            print(f"[I {timestamp}] Trial {self.completed_trials} finished with value: {f1:.4f} and parameters: {params}.", flush=True)

    # Configurar Ray
    ray.init(
        ignore_reinit_error=True,
        include_dashboard=False,
        logging_level=logging.CRITICAL,
        log_to_driver=False
    )

    # Función entrenable para Ray Tune
    def trainable(config):
        model = RandomForestClassifier(
            n_estimators=config["n_estimators"],
            max_depth=config["max_depth"],
            min_samples_split=config["min_samples_split"],
            random_state=42
        )
        model.fit(X_train, y_train)
        y_pred = model.predict(X_test)
        f1 = f1_score(y_test, y_pred)
        train.report({"f1_score": f1})

    # Espacio de búsqueda
    param_space = {
        "n_estimators": tune.randint(10, 200),
        "max_depth": tune.randint(2, 32),
        "min_samples_split": tune.randint(2, 20)
    }

    # Configurar el sintonizador
    tuner = Tuner(
        trainable,
        param_space=param_space,
        tune_config=tune.TuneConfig(
            metric="f1_score",
            mode="max",
            num_samples=n_trials,
            search_alg=BasicVariantGenerator(),
            max_concurrent_trials=4
        ),
        run_config=train.RunConfig(
            verbose=0,
            callbacks=[ProgressCallback(n_trials)]
        )
    )

    # Ejecutar optimización
    print("Iniciando búsqueda de hiperparámetros con Ray Tune...")
    start_time = time.time()
    results = tuner.fit()
    ray_time = time.time() - start_time

    # Obtener resultados
    best_result = results.get_best_result(metric="f1_score", mode="max")
    ray_best_params = best_result.config
    ray_f1 = best_result.metrics["f1_score"]

    # Convertir resultados a DataFrame
    ray_results = []
    for i, result in enumerate(results):
        ray_results.append({
            "trial": i + 1,
            "params": result.config,
            "f1_score": result.metrics["f1_score"]
        })
    ray_results_df = pd.DataFrame(ray_results)

    # Cerrar Ray
    ray.shutdown()

    # Entrenar modelo final con los mejores parámetros
    ray_best_model = RandomForestClassifier(
        n_estimators=ray_best_params["n_estimators"],
        max_depth=ray_best_params["max_depth"],
        min_samples_split=ray_best_params["min_samples_split"],
        random_state=42
    )
    ray_best_model.fit(X_train, y_train)
    y_pred_ray = ray_best_model.predict(X_test)

    return ray_best_model, ray_f1, ray_time, ray_results_df, y_pred_ray, ray_best_params

**Bloque 3:** Creacion de tablas y gráficos para comparación de modelos.

- **`compare_results()`** 
Compara los resultados de tres modelos (base, Optuna y Ray Tune) y genera visualizaciones, incluyendo tablas comparativas, gráficos de rendimiento, evolución de F1-Score y matrices de confusión.

In [ ]:
def compare_results(base_f1, base_time, optuna_f1, optuna_time, ray_f1, ray_time,
                   optuna_results, ray_results_df, y_test,
                   y_pred_base, y_pred_optuna, y_pred_ray, target_names):
    """
    Compara los resultados de tres modelos (base, Optuna y Ray Tune) y genera visualizaciones.

    Incluye:
    - Tabla comparativa de F1-score, tiempo de entrenamiento y número de trials.
    - Gráfico de barras con F1-score y tiempos.
    - Evolución del F1-score en los trials de Optuna y Ray Tune.
    - Matrices de confusión para cada modelo.

    Args:
        base_f1 (float): F1-score del modelo base.
        base_time (float): Tiempo de entrenamiento del modelo base.
        optuna_f1 (float): F1-score del modelo optimizado con Optuna.
        optuna_time (float): Tiempo total de optimización con Optuna.
        ray_f1 (float): F1-score del modelo optimizado con Ray Tune.
        ray_time (float): Tiempo total de optimización con Ray Tune.
        optuna_results (pd.DataFrame): Resultados de los trials de Optuna.
        ray_results_df (pd.DataFrame): Resultados de los trials de Ray Tune.
        y_test (np.ndarray): Etiquetas verdaderas del conjunto de prueba.
        y_pred_base (np.ndarray): Predicciones del modelo base.
        y_pred_optuna (np.ndarray): Predicciones del modelo optimizado con Optuna.
        y_pred_ray (np.ndarray): Predicciones del modelo optimizado con Ray Tune.
        target_names (list): Nombres de las clases objetivo.

    Returns:
        None. Solo imprime y visualiza resultados.
    """
    print("\n" + "="*50)
    print("5. COMPARACIÓN DE RESULTADOS")
    print("="*50)
    
    # Crear tabla comparativa
    comparison_data = {
        'Método': ['Modelo Base', 'Optuna', 'Ray Tune'],
        'F1-score': [base_f1, optuna_f1, ray_f1],
        'Tiempo (s)': [base_time, optuna_time, ray_time],
        'Número de Trials': ['-', len(optuna_results), len(ray_results_df)]
    }
    comparison_df = pd.DataFrame(comparison_data)

    # Mostrar tabla comparativa
    print("\nTabla Comparativa:")
    print("-"*60)
    print(comparison_df.to_string(index=False))
    print("-"*60)


    # Gráficos
    
    # Figura 1: Comparación de métricas
    fig1, axes = plt.subplots(1, 2, figsize=(15, 6))
    
    # Gráfico 1: Comparación de F1-scores
    sns.barplot(x='Método', y='F1-score', data=comparison_df, ax=axes[0], 
                hue='Método', palette="viridis", legend=False, dodge=False)
    axes[0].set_title('Comparación de Rendimiento (F1-score)')
    axes[0].set_ylim(0.9, 1.0)
    for i, v in enumerate(comparison_df['F1-score']):
        axes[0].text(i, v + 0.005, f"{v:.4f}", ha='center', fontsize=12)

    # Gráfico 2: Comparación de tiempos de ejecución
    sns.barplot(x='Método', y='Tiempo (s)', data=comparison_df, ax=axes[1], 
                hue='Método', palette="rocket", legend=False, dodge=False)
    axes[1].set_title('Comparación de Tiempo de Ejecución')
    for i, v in enumerate(comparison_df['Tiempo (s)']):
        axes[1].text(i, v + 0.1, f"{v:.2f}s", ha='center', fontsize=12)


    # Figura 2: Evolución de optimizaciones
    fig2, (ax2, ax3) = plt.subplots(1, 2, figsize=(16, 6))
    
    # Optuna
    ax2.plot(optuna_results['number'], optuna_results['value'], 'o-', color='teal')
    ax2.set_title('Evolución de F1-score en Optuna')
    ax2.set_xlabel('Trial')
    ax2.set_ylabel('F1-score')
    ax2.grid(True)
    best_trial_optuna = optuna_results.loc[optuna_results['value'].idxmax(), 'number']
    ax2.axvline(x=best_trial_optuna, color='r', linestyle='--', alpha=0.7, 
               label=f'Mejor trial: {best_trial_optuna}')
    ax2.legend()
    
    # Ray Tune
    ax3.plot(ray_results_df['trial'], ray_results_df['f1_score'], 'o-', color='purple')
    ax3.set_title('Evolución de F1-score en Ray Tune')
    ax3.set_xlabel('Trial')
    ax3.set_ylabel('F1-score')
    ax3.grid(True)
    best_trial_ray = ray_results_df.loc[ray_results_df['f1_score'].idxmax(), 'trial']
    ax3.axvline(x=best_trial_ray, color='r', linestyle='--', alpha=0.7,
               label=f'Mejor trial: {best_trial_ray}')
    ax3.legend()


    # Figura 3: Matrices de confusión
    fig3, axes = plt.subplots(1, 3, figsize=(18, 5))
    fig3.suptitle('Matrices de Confusión Comparativas', fontsize=16)
    
    # Modelo Base
    cm_base = confusion_matrix(y_test, y_pred_base)
    sns.heatmap(cm_base, annot=True, fmt='d', cmap='Blues', ax=axes[0],
                xticklabels=target_names, 
                yticklabels=target_names)
    axes[0].set_title('Modelo Base')
    axes[0].set_ylabel('Real')
    axes[0].set_xlabel('Predicho')
    
    # Optuna
    cm_optuna = confusion_matrix(y_test, y_pred_optuna)
    sns.heatmap(cm_optuna, annot=True, fmt='d', cmap='Greens', ax=axes[1],
                xticklabels=target_names, 
                yticklabels=target_names)
    axes[1].set_title('Optuna Optimizado')
    axes[1].set_xlabel('Predicho')
    axes[1].set_ylabel('')
    
    # Ray Tune
    cm_ray = confusion_matrix(y_test, y_pred_ray)
    sns.heatmap(cm_ray, annot=True, fmt='d', cmap='Purples', ax=axes[2],
                xticklabels=target_names, 
                yticklabels=target_names)
    axes[2].set_title('Ray Tune Optimizado')
    axes[2].set_xlabel('Predicho')
    axes[2].set_ylabel('')

    plt.tight_layout()
    plt.subplots_adjust(top=0.85)

    # Guardar las figuras
    fig1.savefig("comparacion_metricas.png", dpi=300, bbox_inches='tight')
    fig2.savefig("evolucion_optimizadores.png", dpi=300, bbox_inches='tight')
    fig3.savefig("matrices_confusion.png", dpi=300, bbox_inches='tight')

    plt.show()

**Bloque 4:** Funciones de ejecución.

Debido a que Ray Tune produce errores como borrar las celdas de salida previas a us ejecución, la función main fue segmentada en dos partes:

- **`main()`**
Función que ejecuta el flujo: carga de datos, entrenamiento, creación de modelo base y optimizados con Optuna y Ray Tune.

- **`main_2()`**
Función que toma los resultados obtenidos por main() y muestra los resultados obtenidos a partir de la comparación de los tres modelos.

In [ ]:
def main():
    """
    Ejecuta el flujo completo de entrenamiento y optimización del modelo.

    Incluye:
    - Configuración del entorno y estilos.
    - Carga y preprocesamiento del dataset de cáncer de mama.
    - Entrenamiento de un modelo base sin tuning.
    - Optimización de hiperparámetros con Optuna y Ray Tune.
    - Evaluación de desempeño (F1-score y tiempo).

    Returns:
        tuple: Variables necesarias para reportes y comparación posterior. Incluye:
            - info_df (pd.DataFrame): Resumen del dataset.
            - optuna_best_params (dict): Mejores parámetros de Optuna.
            - ray_best_params (dict): Mejores parámetros de Ray Tune.
            - base_f1 (float): F1-score del modelo base.
            - base_time (float): Tiempo de entrenamiento del modelo base.
            - optuna_f1 (float): F1-score con Optuna.
            - optuna_time (float): Tiempo de optimización con Optuna.
            - best_trial (int): ID del mejor trial de Optuna.
            - ray_f1 (float): F1-score con Ray Tune.
            - ray_time (float): Tiempo de optimización con Ray Tune.
            - optuna_results (pd.DataFrame): Resultados de Optuna.
            - ray_results (pd.DataFrame): Resultados de Ray Tune.
            - y_test (np.ndarray): Etiquetas reales del test.
            - y_pred_base (np.ndarray): Predicciones del modelo base.
            - y_pred_optuna (np.ndarray): Predicciones del modelo optimizado con Optuna.
            - y_pred_ray (np.ndarray): Predicciones del modelo optimizado con Ray Tune.
            - target_names (np.ndarray): Nombres de las clases objetivo.
    """
    # 0. Configuración inicial
    setup_environment()
    
    # 1. Carga y preprocesamiento de datos
    data = load_breast_cancer()
    X_train, X_test, y_train, y_test, data, info_df = load_and_preprocess_data(data)
    target_names = data.target_names
    
    # 2. Modelo base (sin tuning)
    base_model, base_f1, base_time, y_pred_base = train_baseline_model(
        X_train, y_train, X_test, y_test, target_names
    )

    # 3. Optimización con Ray Tune (20 trials). Se ejecuta primero para evitar problemas de memoria con Optuna.
    ray_model, ray_f1, ray_time, ray_results, y_pred_ray, ray_best_params = optimize_with_ray_tune(
        X_train, y_train, X_test, y_test, n_trials=20
    )
    
    # 4. Optimización con Optuna (20 trials)
    optuna_model, optuna_f1, optuna_time, optuna_results, y_pred_optuna, optuna_best_params, best_trial = optimize_with_optuna(
        X_train, y_train, X_test, y_test, n_trials=20
    )

    # Mensajes finales antes de retornar
    print("\nDatos cargados y preprocesados correctamente.")
    print(f"Modelo base entrenado sin optimización en {base_time:.2f} segundos.")
    print(f"Modelo optimizado con Optuna completado en {optuna_time:.2f} segundos.")
    print(f"Modelo optimizado con Ray Tune completado en {ray_time:.2f} segundos.")

    return (info_df, optuna_best_params, ray_best_params, base_f1, base_time,
           optuna_f1, optuna_time, best_trial, ray_f1, ray_time,
           optuna_results, ray_results, y_test,
           y_pred_base, y_pred_optuna, y_pred_ray, target_names)
        
    
def main_2(info_df, optuna_best_params, ray_best_params, base_f1, base_time,
           optuna_f1, optuna_time, best_trial, ray_f1, ray_time,
           optuna_results, ray_results, y_test,
           y_pred_base, y_pred_optuna, y_pred_ray, target_names):
    """
    Segunda parte del flujo: resumen de resultados, comparación y visualización.

    Esta función:
    - Muestra resumen del dataset.
    - Imprime mejores hiperparámetros encontrados.
    - Reporta desempeño en F1-score y tiempo de entrenamiento.
    - Compara gráficamente los métodos (modelo base, Optuna y Ray Tune).
    - Guarda los resultados finales en un archivo CSV.

    Args:
        info_df (pd.DataFrame): Resumen del dataset.
        optuna_best_params (dict): Mejores parámetros de Optuna.
        ray_best_params (dict): Mejores parámetros de Ray Tune.
        base_f1 (float): F1-score del modelo base.
        base_time (float): Tiempo de entrenamiento del modelo base.
        optuna_f1 (float): F1-score del modelo optimizado con Optuna.
        optuna_time (float): Tiempo total de optimización con Optuna.
        best_trial (int): Trial con mejor resultado en Optuna.
        ray_f1 (float): F1-score del modelo optimizado con Ray Tune.
        ray_time (float): Tiempo total de optimización con Ray Tune.
        optuna_results (pd.DataFrame): Resultados de los trials de Optuna.
        ray_results (pd.DataFrame): Resultados de los trials de Ray Tune.
        y_test (np.ndarray): Etiquetas reales del conjunto de prueba.
        y_pred_base (np.ndarray): Predicciones del modelo base.
        y_pred_optuna (np.ndarray): Predicciones del modelo Optuna.
        y_pred_ray (np.ndarray): Predicciones del modelo Ray Tune.
        target_names (np.ndarray): Nombres de las clases objetivo.

    Returns:
        None. Imprime, visualiza y guarda resultados.
    """
    # Mostrar info del dataset en tabla
    print("Resumen del dataset:")
    info_df = info_df.T  # Transpone el DataFrame
    info_df.columns = ["Valor"]  # Renombra la única columna resultante
    print(info_df)

    # Mostrar mejores parámetros
    print("\nMejores parámetros encontrados por Optuna:")
    for param, value in optuna_best_params.items():
        print(f"- {param}: {value}")

    print("\nMejores parámetros encontrados por Ray Tune:")
    for param, value in ray_best_params.items():
        print(f"- {param}: {value}")

    # Mostrar resultados de cada modelo
    print("\nResultados del modelo base:")
    print(f"- F1-score: {base_f1:.4f}")
    print(f"- Tiempo de entrenamiento: {base_time:.2f} segundos")
    
    print("\nResultados del modelo optimizado con Optuna:")
    print(f"- F1-score: {optuna_f1:.4f}")
    print(f"- Tiempo total de optimización: {optuna_time:.2f} segundos")
    print(f"- Mejor trial: #{best_trial}")

    print("\nResultados del modelo optimizado con Ray Tune:")
    print(f"- F1-score: {ray_f1:.4f}")
    print(f"- Tiempo total de optimización: {ray_time:.2f} segundos")

    # Comparación de hiperparámetros
    param_comparison = pd.DataFrame({
        'Parámetro': list(optuna_best_params.keys()),
        'Optuna': list(optuna_best_params.values()),
        'Ray Tune': [ray_best_params[k] for k in optuna_best_params.keys()]
    })

    print("\nComparación de Hiperparámetros:")
    print(param_comparison.to_string(index=False))

    mejor_metodo = 'Optuna' if optuna_f1 >= ray_f1 and optuna_f1 >= base_f1 else 'Ray Tune' if ray_f1 > base_f1 else 'Modelo Base'
    print(f"\nMejor modelo: {mejor_metodo} (F1-score: {max(base_f1, optuna_f1, ray_f1):.4f})")


    # 5. Comparación de resultados (con parámetros adicionales)
    compare_results(
        base_f1, base_time, 
        optuna_f1, optuna_time, 
        ray_f1, ray_time,
        optuna_results, 
        ray_results,
        y_test,
        y_pred_base,
        y_pred_optuna,
        y_pred_ray,
        target_names
    )
    
    # # Guardar resultados finales
    # final_results = pd.DataFrame({
    #     'Método': ['Modelo Base', 'Optuna', 'Ray Tune'],
    #     'F1-score': [base_f1, optuna_f1, ray_f1],
    #     'Tiempo (s)': [base_time, optuna_time, ray_time],
    #     'Trials': [0, 20, 20]
    # })
    # final_results.to_csv('resultados_finales.csv', index=False)
    # print("Resultados guardados en 'resultados_finales.csv'")

# 4. Visualización de resultados

Para evitar problemas en las celdas de salida, la ejecución de realiza en dos pasos, en dos celdas de codigo distintas.

---

- **`Primer paso`**
Almacenar en una variable todos los resultados generados por los modelos a comparar.

- **`Segundo paso`**
Usar los resultados obtenidos con main() para mostrarlos junto a las comparaciones de modelos.

In [ ]:
# Primer paso
resultados = main()

In [ ]:
# Segundo paso
main_2(*resultados)

# 5. Análisis de reultados y reflexiones Finales

- La comparación entre un modelo Random Forest base y sus versiones optimizadas mediante Optuna y Ray Tune permitió evaluar el impacto del tuning de hiperparámetros en el rendimiento predictivo sobre el dataset de cáncer de mama.

- Principales conclusiones:

- El modelo base, sin optimización, ya presenta un buen desempeño con un F1-score de 0.9488, confirmando que Random Forest es robusto aún con parámetros por defecto.

    - La optimización con Optuna logró el mejor F1-score (0.9585) con un tiempo de cómputo razonable (3.11 segundos), mostrando una excelente relación costo-beneficio.

    - Ray Tune también mejoró el rendimiento frente al modelo base (0.9537), aunque con un tiempo de ejecución significativamente mayor (40 segundos), lo que podría no ser justificable en tareas donde los recursos o el tiempo son limitados.

    - Ambos métodos coincidieron en que min_samples_split = 3 era óptimo, aunque difirieron en la profundidad y número de árboles, lo que muestra cómo distintas estrategias pueden converger parcialmente.

### Reflexión:
- La optimización de hiperparámetros puede entregar mejoras importantes en modelos ya sólidos como Random Forest. Sin embargo, el tiempo de optimización y la eficiencia computacional deben considerarse al elegir la herramienta adecuada. En este caso, Optuna se presenta como la mejor opción, combinando precisión, velocidad y simplicidad.